# OpenMP Directive Syntax

OpenMP *directives* use base-language mechanisms to specify OpenMP program behavior. In C code, the directives are formed exclusively with pragmas, whereas in C++ code, directives are formed from either pragmas or attributes. Fortran directives are formed with comments in free form and fixed form sources (codes). All of these mechanisms allow the compilation to ignore the OpenMP directives if OpenMP is not supported or enabled.

The OpenMP directive is a combination of the base-language mechanism and a  _directive-specification_ , as shown below. The  _directive-specification_  consists of the  _directive-name_  which may seldomly have arguments,  followed by optional  _clauses_ . Full details of the syntax can be found in the OpenMP Specification. Illustrations of the syntax is given in the examples.

The formats for combining a base-language mechanism and a  _directive-specification_  are:

C/C++ pragmas  

&nbsp;&nbsp;&nbsp;&nbsp;
__\#pragma omp__
*directive\-specification*


C++ attributes  
&nbsp;&nbsp;&nbsp;&nbsp;
__[[omp :: directive\(__
_directive-specification_
__\)]]__

&nbsp;&nbsp;&nbsp;&nbsp;
__[[using omp : directive\(__
_directive-specification_
__\)]]__

Fortran comments  
&nbsp;&nbsp;&nbsp;&nbsp;__!\$omp__  _directive-specification_

where __c\\$omp__ and __*\$omp__ may be used in Fortran fixed form sources.

Most OpenMP directives accept clauses that alter the semantics of the directive in some way,  and some directives also accept parenthesized arguments that follow the directive name.  A clause may just be a keyword (e.g., __untied__) or it may also accept argument lists  (e.g., __shared(x,y,z)__) and/or optional modifiers (e.g., __tofrom__ in  __map(tofrom: x,y,z)__). Clause modifiers may be "simple" or "complex" -- a complex modifier consists of a  keyword followed by one or more parameters, bracketed by parentheses, while a simple  modifier does not. An example of a complex modifier is the __iterator__ modifier,  as in __map(iterator(i=0:n),tofrom: p[i])__, or the __step__ modifier, as in __linear(x:__ __ref,__ __step(4))__.  In the preceding examples, __tofrom__ and __ref__ are simple modifiers.

## C/C++ Pragmas

OpenMP C and C++ directives can be specified with the C/C++ __#pragma__ directive. An OpenMP directive begins with __#pragma omp__ and is followed by the  OpenMP directive name, and required and optional clauses. Lines are continued in the  usual manner, and comments may be included at the end. Directives are case sensitive.

The example below illustrates the use of the OpenMP pragma form. The first pragma (PRAG 1) specifies a combined __parallel for__ directive, with a __num_threads__ clause, and a comment.  The second pragma (PRAG 2) shows the same directive split across two lines. The next nested pragmas (PRAG 3 and 4) show the previous combined directive as  two separate directives. The executable directives above all apply to the next  statement. The __parallel__ directive can be applied to a  _structured_   _block_  as shown in PRAG 5.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name:       directive_syntax_pragma.1
* type:       C
*/
#include   <omp.h>
#include <stdio.h>
#define NT 4
#define thrd_no omp_get_thread_num

int main(){
    #pragma omp parallel for num_threads(NT)                 // PRAG 1
    for(int i=0; i<NT; i++) printf("thrd no %d\n",thrd_no());

    #pragma omp parallel for \
                num_threads(NT)                              // PRAG 2
    for(int i=0; i<NT; i++) printf("thrd no %d\n",thrd_no());

    #pragma omp parallel num_threads(NT)                     // PRAG 3-4
    #pragma omp for
    for(int i=0; i<NT; i++) printf("thrd no %d\n",thrd_no());

    #pragma omp parallel num_threads(NT)                     // PRAG 5
    {
       int no = thrd_no();
       if (no%2) { printf("thrd no %d is Odd \n",no);}
       else      { printf("thrd no %d is Even\n",no);}

       #pragma omp for
       for(int i=0; i<NT; i++) printf("thrd no %d\n",thrd_no());
    }
}
/*
      repeated 4 times, any order
      OUTPUT: thrd no 0
      OUTPUT: thrd no 1
      OUTPUT: thrd no 2
      OUTPUT: thrd no 3

      any order
      OUTPUT: thrd no 0 is Even
      OUTPUT: thrd no 2 is Even
      OUTPUT: thrd no 1 is Odd
      OUTPUT: thrd no 3 is Odd
*/



## C++ Attributes

**directive syntax!attribute, C++** **attribute syntax, C++**

OpenMP directives for C++ can also be specified with  the __directive__ extension for the C++11 standard  _attributes_ .

The C++ example below shows two ways to parallelize a __for__ loop using the __#pragma__ syntax. The first pragma uses the combined __parallel__ __for__ directive, and the second applies the uncombined closely nested directives, __parallel__ and __for__, directly to the same statement.  These are labeled PRAG 1-3.

Using the attribute syntax, the same construct in PRAG 1 is applied two different ways in attribute form, as shown in the ATTR 1 and ATTR 2 sections. In ATTR 1 the attribute syntax is used with the __omp ::__ namespace form. In ATTR 2 the attribute syntax is used with the __using omp :__ namespace form.

Next, parallelization is attempted by applying directives using two different syntaxes. For ATTR 3 and PRAG 4, the loop parallelization will fail to compile because multiple directives that apply to the same statement must all use either the attribute syntax or the pragma syntax. The lines have been commented out and labeled INVALID.

While multiple attributes may be applied to the same statement, compilation may fail if the ordering of the directive matters. For the ATTR 4-5 loop parallelization, the __parallel__ directive precedes  the __for__ directive, but the compiler may reorder consecutive attributes. If the directives are reversed, compilation will fail.

The attribute directive of the ATTR 6 section resolves the previous problem (in ATTR 4-5). Here, the __sequence__ attribute is used to apply ordering to the directives of ATTR 4-5, using the __omp__ __::__ namespace qualifier. (The __using omp :__ namespace form is not available for the __sequence__ attribute.)  Note, for the __sequence__ attribute a comma must separate the __directive__ extensions.

The last 3 pairs of sections (PRAG DECL 1-2, 3-4, and 5-6) show cases where  directive ordering does not matter for __declare__ __simd__ directives.

In section PRAG DECL 1-2, the two loops use different SIMD forms of the  _P_  function (one with __simdlen(4)__ and the other with __simdlen(8)__),  as prescribed by the two different __declare__ __simd__ directives applied to the  _P_  function definitions (at the beginning of the code).  The directives use the pragma syntax, and order is not important.  For the next set of loops  (PRAG DECL 3-4) that use the  _Q_  function, the attribute syntax is  used for the __declare__ __simd__ directives.  The result is compliant code since directive order is irrelevant. Sections ATTR DECL 5-6 are included for completeness. Here, the attribute  form of the __simd__ directive is used for loops calling the  _Q_  function,  in combination with the attribute form of the __declare__ __simd__  directives declaring the variants for  _Q_ .

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name:       directive_syntax_attribute.1
* type:       C++
* version:    omp_5.0
*/
#include <stdio.h>
#include <omp.h>
#define NT 4
#define thrd_no omp_get_thread_num

#pragma omp declare simd linear(i) simdlen(4)
#pragma omp declare simd linear(i) simdlen(8)
double P(int i){ return (double)i * (double)i; }

[[omp::directive(declare simd linear(i) simdlen(4))]]
[[omp::directive(declare simd linear(i) simdlen(8))]]
double Q(int i){ return (double)i * (double)i; }

int main() {

    #pragma omp parallel for num_threads(NT)                 // PRAG 1
    for(int i=0; i<NT; i++) printf("thrd no %d\n",thrd_no());

    #pragma omp parallel num_threads(NT)                     // PRAG 2
    #pragma omp for                                          // PRAG 3
    for(int i=0; i<NT; i++) printf("thrd no %d\n",thrd_no());

                                                             // ATTR 1
    [[omp::directive( parallel for num_threads(NT))]]
    for(int i=0; i<NT; i++) printf("thrd no %d\n",thrd_no());

                                                             // ATTR 2
    [[using omp : directive( parallel for num_threads(NT))]]
    for(int i=0; i<NT; i++) printf("thrd no %d\n",thrd_no());

 // INVALID-- attribute and non-attribute on same statement
 // [[ omp :: directive( parallel num_threads(NT) ) ]]          ATTR 3
 // #pragma omp for                                             PRAG 4
 // for(int i=0; i<NT; i++) printf("thrd no %d\n",thrd_no());


 // INVALID-- directive order not guaranteed
 // [[ omp :: directive( parallel num_threads(NT) ) ]]          ATTR 4
 // [[ omp :: directive( for                      ) ]]          ATTR 5
 // for(int i=0; i<NT; i++) printf("thrd no %d\n",thrd_no());

                                                             // ATTR 6
    [[omp::sequence(directive(parallel num_threads(NT)),directive(for))]]
    for(int i=0; i<NT; i++) printf("thrd no %d\n",thrd_no());

    double tmp=0.0f;
    #pragma omp simd reduction(+:tmp) simdlen(4)
    for(int i=0;i<100;i++) tmp += P(i);                 // PRAG DECL 1
    #pragma omp simd reduction(+:tmp) simdlen(8)
    for(int i=0;i<100;i++) tmp += P(i);                 // PRAG DECL 2
    printf("%f\n",tmp);

    tmp=0.0f;
    #pragma omp simd reduction(+:tmp) simdlen(4)
    for(int i=0;i<100;i++) tmp += Q(i);                 // ATTR DECL 3
    #pragma omp simd reduction(+:tmp) simdlen(8)
    for(int i=0;i<100;i++) tmp += Q(i);                 // ATTR DECL 4
    printf("%f\n",tmp);

    tmp=0.0f;
    [[ omp :: directive(simd reduction(+:tmp) simdlen(4))]]
    for(int i=0;i<100;i++) tmp += Q(i);                 // ATTR DECL 5
    [[ omp :: directive(simd reduction(+:tmp) simdlen(8))]]
    for(int i=0;i<100;i++) tmp += Q(i);                 // ATTR DECL 6
    printf("%f\n",tmp);
}
//    repeated 5 times, any order:
//    OUTPUT: thrd no  0
//    OUTPUT: thrd no  1
//    OUTPUT: thrd no  2
//    OUTPUT: thrd no  3

//    repeated 3 time:
//    OUTPUT: 656700.000000




## Fortran Comments (Fixed Source Form)

OpenMP directives in Fortran codes with fixed source form are specified as comments with one of the __!\\$omp__, __c\\$omp__, and __*\\$omp__ sentinels, followed by a directive name, and required and optional clauses.  The sentinel must begin in column 1.

In the example below the first directive (DIR 1) specifies the __parallel__ __do__ combined directive, with a __num_threads__ clause, and a comment. The second directive (DIR 2) shows the same directive split across two lines. The next nested directives (DIR 3 and 4) show the previous combined directive as two separate directives.  Here, an __end__ directive (__end__ __parallel__) must be specified to demarcate the range (region) of the __parallel__ directive.

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: directive_syntax_F_fixed_comment.1
! type: F-fixed
      program main
      include 'omp_lib.h'
      integer NT

      NT =4

c     sentinel c\$omp or *\$omp can also be used

c$omp parallel do num_threads(NT) !comments allowed here   DIR 1
      do i = 1,NT
        write(*,'("thrd no", i2)') omp_get_thread_num()
      end do

!$omp parallel do
!$omp+ num_threads(NT)          !cont. w. char in col. 6   DIR 2
      do i = 1,NT
        write(*,'("thrd no", i2)') omp_get_thread_num()
      end do

*$omp parallel num_threads(NT)  !multi-directive form      DIR 3
*$omp do                        !                          DIR 4
      do i = 1,NT
         write(*,'("thrd no", i2)') omp_get_thread_num()
      end do
*$omp end parallel
      end
!     repeated 3 times, any order
!     OUTPUT: thrd no  0
!     OUTPUT: thrd no  1
!     OUTPUT: thrd no  2
!     OUTPUT: thrd no  3



## Fortran Comments (Free Source Form)

OpenMP directives in Fortran codes with free source form are specified as comments that use the `!$omp` sentinel, followed by the directive name, and required and optional clauses.  Lines are continued with an ending ampersand (__&__), and the continued line begins with `!$omp` or `!$omp&`. Comments may appear on the same line as the directive.  Directives are case insensitive.

In the example below the first directive (DIR 1) specifies the __parallel__ __do__ combined directive, with a __num_threads__ clause, and a comment. The second directive (DIR 2) shows the same directive split across two lines.  The next nested directives (DIR 3 and 4) show the previous combined directive as two separate directives.  Here, an __end__ directive (__end__ __parallel__) must be specified to demarcate the range (region) of the __parallel__ directive.

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name:       directive_syntax_F_free_comment.1
! type:       F-free
  program main
     use omp_lib
     integer,parameter :: NT = 4

     !$omp parallel do num_threads(NT)                !DIR 1
     do i = 1,NT
       write(*,'("thrd no", i2)') omp_get_thread_num()
     end do

     !$omp  parallel do  &     !continue line         !DIR 2
     !$omp num_threads(NT)     !or !$omp&
     do i = 1,NT
       write(*,'("thrd no", i2)') omp_get_thread_num()
     end do

     !$omp parallel num_threads(NT)                   !DIR 3
     !$omp do                                         !DIR 4
     do i = 1,NT
        write(*,'("thrd no", i2)') omp_get_thread_num()
     end do
     !$omp end parallel

  end program

!     repeated 3 times, any order
!     OUTPUT: thrd no  0
!     OUTPUT: thrd no  1
!     OUTPUT: thrd no  2
!     OUTPUT: thrd no  3



As of OpenMP 5.1, __block__ and __end__ __block__ statements can be used to designate  a structured block for an OpenMP region, and any paired OpenMP __end__ directive becomes optional, as shown in the next example.  Note, the variables  _i_  and  _thrd_no_  are declared within the  block structure and are hence private. It was necessary to explicitly declare the  _i_  variable, due to the __implicit none__ statement;  it could have also been declared outside the structured block.

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name:       directive_syntax_F_block.1
! type:       F-free
! version:    omp_5.1
  program main

     use omp_lib
     implicit none
     integer,parameter :: NT = 2, chunks=3

    !$omp parallel num_threads(NT)
     block                          ! Fortran 2008 OMP 5.1
       integer :: thrd_no,i
       thrd_no= omp_get_thread_num()
        !$omp do schedule(static,chunks)
        do i = 1,NT*chunks
           write(*,'("ndx=",i0.2," thrd_no=", i0.2)') i,thrd_no
        end do
     end block
  end program

! any order
! OUTPUT: ndx=01 thrd_no=00
! OUTPUT: ndx=02 thrd_no=00
! OUTPUT: ndx=03 thrd_no=00
! OUTPUT: ndx=04 thrd_no=01
! OUTPUT: ndx=05 thrd_no=01
! OUTPUT: ndx=06 thrd_no=01

A Fortran BLOCK construct may eliminate the need for a paired __end__ directive for an OpenMP construct,  as illustrated in the following example.

The first __parallel__ construct is specified with an OpenMP loosely structured block  (where the first executable construct is not a Fortran 2008 BLOCK construct).  A paired __end__ directive must end the OpenMP construct. The second __parallel__ construct is specified with an OpenMP strictly structured block  (consists only of a single Fortran BLOCK construct).  The paired __end__ directive is optional in this case, and is not used here.

The next two __parallel__ directives form an enclosing outer __parallel__ construct  and a nested inner __parallel__ construct. The first __end__ __parallel__ directive that subsequently appears terminates the inner __parallel__ construct,  because a paired __end__ directive immediately following a BLOCK construct that is  a strictly structured block of an OpenMP construct is treated as the terminating end directive  of that construct.  The next __end__ __parallel__ directive is required to terminate the outer __parallel__ construct.

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name:       directive_syntax_F_block.2
! type:       F-free
! version:    omp_5.1
program main

   use omp_lib
   implicit none

  !$omp parallel num_threads(2)
     if( omp_get_thread_num() == 0 ) &
         print*, "Loosely  structured block  -- end required."
     block                                ! BLOCK Fortran 2008
       if( omp_get_thread_num() == 0 ) &
           print*, "                           --"
     end block
  !$omp end parallel

  !$omp parallel num_threads(2)
     block
       if( omp_get_thread_num() == 0 ) &
           print*, "Strictly structured block  -- end not required."
     end block
 !!$omp end parallel !is optional for strictly structured block

  print*, "Sequential part"

  !$omp parallel num_threads(2)                      !outer parallel
     if( omp_get_thread_num() == 0 ) &
         print*, "Outer, loosely  structured block."
     !$omp parallel num_threads(2)                   !inner parallel
        block
          if( omp_get_thread_num() == 0 ) &
          print*, "Inner, strictly structured block."
        end block
     !$omp end parallel
  !$omp end parallel
  ! Two end directives are required here.
  ! A single "!$omp end parallel" terminator will fail.
  ! 1st end directive is assumed to be for inner parallel construct.
  ! 2nd end directive applies to outer parallel construct.

end program

!OUTPUT, in order:
! Loosely  structured block  -- end required.
!                            --
! Strictly structured block  -- end not required.
! Sequential part
! Outer, loosely  structured block.
! Inner, strictly structured block.
! Inner, strictly structured block.

